In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
  
import tensorflow as tf
if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 4578792217134177669
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 11320098816
locality {
  bus_id: 1
  links {
  }
}
incarnation: 14771846386673778380
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"
xla_global_id: 416903419
]
Default GPU Device: /device:GPU:0


In [ ]:
import platform
import tensorflow as tf

if tf.test.is_built_with_cuda() == False:
    print('ARBEIT OHNE GPU')

print(platform.machine(), '\n')
print(platform.version(), '\n')
print(platform.platform(), '\n')
print(platform.uname(), '\n')
print(platform.system(), '\n')
print(platform.processor())

x86_64 

#1 SMP Sat Jun 5 09:50:34 PDT 2021 

Linux-5.4.104+-x86_64-with-Ubuntu-18.04-bionic 

uname_result(system='Linux', node='165fc4fee9ed', release='5.4.104+', version='#1 SMP Sat Jun 5 09:50:34 PDT 2021', machine='x86_64', processor='x86_64') 

Linux 

x86_64


In [ ]:
import pandas as pd
import numpy as np
import json
import re
import string

df_drive = pd.read_csv('/content/drive/MyDrive/NLP for Web/fullset.csv')

In [ ]:
from sklearn.preprocessing import LabelEncoder

lb_make = LabelEncoder()
label_id = lb_make.fit_transform(df_drive.label)

df_drive['label_id'] = label_id
df_drive.dropna(inplace=True)

In [ ]:
from sklearn.model_selection  import train_test_split

df_train, df_test = train_test_split(df_drive, test_size=0.2, random_state=42)

df_train.reset_index(inplace=True, drop=True)
df_test.reset_index(inplace=True, drop=True)

In [ ]:
df_test.to_csv('testset.csv', index=False)

In [ ]:
!pip install transformers

## Load Bert uncased Model

In [ ]:
from transformers import AutoTokenizer, TFAutoModel

#Downloading the tokenizer and the Albert model for fine tuning

tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')
bert=TFAutoModel.from_pretrained('bert-base-cased')

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
# max length of berttokenizer  is 512
max_length=512

Xids_train=np.zeros((df_train.shape[0],max_length))
Xmask_train=np.zeros((df_train.shape[0],max_length))
y_train=np.zeros((df_train.shape[0],1))

Xids_test=np.zeros((df_test.shape[0],max_length))
Xmask_test=np.zeros((df_test.shape[0],max_length))

In [ ]:
for i,sequence in enumerate(df_train['tweet']):
    tokens=tokenizer.encode_plus(sequence,max_length=max_length,padding='max_length',add_special_tokens=True,
                           truncation=True,return_token_type_ids=False,return_attention_mask=True,
                           return_tensors='tf')

    Xids_train[i,:] = tokens['input_ids']
    Xmask_train[i,:] = tokens['attention_mask']
    y_train[i,0] = df_train.loc[i,'label_id']
    
for i,sequence in enumerate(df_test['tweet']):
    tokens=tokenizer.encode_plus(sequence,max_length=max_length,padding='max_length',add_special_tokens=True,
                           truncation=True,return_token_type_ids=False,return_attention_mask=True,
                           return_tensors='tf')
    
    Xids_test[i,:] = tokens['input_ids']
    Xmask_test[i,:] = tokens['attention_mask']

## Define batches

In [ ]:
dataset=tf.data.Dataset.from_tensor_slices((Xids_train,Xmask_train,y_train))

def map_func(input_ids,mask,labels):
    return {'input_ids':input_ids,'attention_mask':mask},labels

dataset=dataset.map(map_func)
dataset=dataset.shuffle(100000).batch(64).prefetch(1000)

DS_size=len(list(dataset))

train=dataset.take(round(DS_size*0.85))
val=dataset.skip(round(DS_size*0.85))

In [ ]:
dataset_test=tf.data.Dataset.from_tensor_slices((Xids_test,Xmask_test))

def map_func(input_ids,mask):
    return {'input_ids':input_ids,'attention_mask':mask}

dataset_test=dataset_test.map(map_func)
# batching it to or the predictions will be multiplied by the shape
dataset_test=dataset_test.batch(64).prefetch(1000)

## Define Model

In [ ]:
#save model
from __future__ import absolute_import, division, print_function
import os

checkpoint_path = "/content/drive/MyDrive/NLP for Web/model"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, 
                                                 save_weights_only=True,
                                                 verbose=1)

In [ ]:
import tensorflow as tf 

def CovFake_model():
  input_ids=tf.keras.layers.Input(shape=(max_length,),name='input_ids',dtype='int32')
  input_mask=tf.keras.layers.Input(shape=(max_length,),name='attention_mask',dtype='int32')
  
  embedding=bert(input_ids,attention_mask=input_mask)[0]
  x=tf.keras.layers.GlobalMaxPool1D()(embedding)
  x=tf.keras.layers.GlobalAveragePooling1D()(embedding)
  x=tf.keras.layers.BatchNormalization()(x)
  x=tf.keras.layers.Dense(256,activation='relu')(x)
  x=tf.keras.layers.Dropout(0.3)(x)
  x=tf.keras.layers.Dense(128,activation='relu')(x)
  x=tf.keras.layers.Dropout(0.3)(x)
  x=tf.keras.layers.Dense(64,activation='relu')(x)
  output=tf.keras.layers.Dense(1,activation='sigmoid')(x)
  
  model=tf.keras.Model(inputs=[input_ids,input_mask],outputs=output)
  
  return model

In [ ]:
model = CovFake_model()

In [ ]:
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 512)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 512)]        0           []                               
                                                                                                  
 tf_bert_model (TFBertModel)    TFBaseModelOutputWi  108310272   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 512,                                         

## Train Model

In [ ]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(),
              optimizer='adam',metrics=[tf.keras.metrics.AUC()])

model.fit(train,validation_data=val,epochs=5, callbacks=[cp_callback])

Epoch 1/5
153/153 [==============================] - ETA: 0s - loss: 0.2047 - auc_2: 0.9727
Epoch 00001: saving model to /content/drive/MyDrive/NLP for Web/model
153/153 [==============================] - 983s 6s/step - loss: 0.2047 - auc_2: 0.9727 - val_loss: 0.2566 - val_auc_2: 0.9933
Epoch 2/5
153/153 [==============================] - ETA: 0s - loss: 0.1316 - auc_2: 0.9881
Epoch 00002: saving model to /content/drive/MyDrive/NLP for Web/model
153/153 [==============================] - 960s 6s/step - loss: 0.1316 - auc_2: 0.9881 - val_loss: 0.1184 - val_auc_2: 0.9962
Epoch 3/5
153/153 [==============================] - ETA: 0s - loss: 0.1199 - auc_2: 0.9901
Epoch 00003: saving model to /content/drive/MyDrive/NLP for Web/model
153/153 [==============================] - 964s 6s/step - loss: 0.1199 - auc_2: 0.9901 - val_loss: 0.0668 - val_auc_2: 0.9979
Epoch 4/5
153/153 [==============================] - ETA: 0s - loss: 0.1041 - auc_2: 0.9925
Epoch 00004: saving model to /content/drive/